<a href="https://colab.research.google.com/github/yurikanade9/datab/blob/main/%EC%8B%A4%EC%8A%B501_%EC%8B%9C%EC%B9%B4%EA%B3%A0%EA%B3%B5%EB%A6%BD%ED%95%99%EA%B5%90_%EB%B0%B0%ED%8F%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# 실제 데이터 세트를 사용하여 SQL과 Python으로 작업하기  

소요 예상 시간: 30분  

## 학습 목표

이 실습을 완료한 후, 다음을 수행할 수 있다:  

* 시카고 공립학교 수준 성과 데이터 세트를 이해하기  
* SQLite 데이터베이스에 데이터 세트를 저장하기  
* 테이블과 열에 대한 메타데이터를 검색하고 혼합 대소문자 열에서 데이터를 쿼리하기  
* 데이터베이스 내장 함수를 활용한 SQL 실습 문제 해결하기  


## Chicago Public Schools - Progress Report Cards (2011-2012)

시카고 시에서는 2011-2012 학년도 학교 보고서 카드를 생성하는 데 사용된 모든 학교 수준 성과 데이터를 보여주는 데이터 세트를 공개하였다. 해당 데이터 세트는 시카고 데이터 포털에서 확인할 수 있다: [https://data.cityofchicago.org/Education/Chicago-Public-Schools-Progress-Report-Cards-2011-/9xs2-f89t](https://data.cityofchicago.org/Education/Chicago-Public-Schools-Progress-Report-Cards-2011-/9xs2-f89t?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkDB0201ENSkillsNetwork20127838-2021-01-01&cm_mmc=Email_Newsletter-\_-Developer_Ed%2BTech-\_-WW_WW-\_-SkillsNetwork-Courses-IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork-20127838&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ)

이 데이터 세트에는 많은 수의 지표가 포함되어 있다. 데이터베이스에 포함된 지표 유형에 익숙해지는 것부터 시작하라: [https://data.cityofchicago.org/api/assets/AAD41A13-BE8A-4E67-B1F5-86E711E09D5F?download=true](https://data.cityofchicago.org/api/assets/AAD41A13-BE8A-4E67-B1F5-86E711E09D5F?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkDB0201ENSkillsNetwork20127838-2021-01-01&download=true&cm_mmc=Email_Newsletter-\_-Developer_Ed%2BTech-\_-WW_WW-\_-SkillsNetwork-Courses-IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork-20127838&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ)

**참고**:

데이터 세트를 시카고 포털에서 직접 다운로드하지 말고, 더 데이터베이스 친화적인 static copy 를 이 <a href="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/FinalModule_Coursera_V5/data/ChicagoPublicSchools.csv?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkDB0201ENSkillsNetwork20127838-2021-01-01">링크</a>에서 다운로드하라.

이제 데이터 일부 내용을 검토하라.



### 데이터베이스에 연결하기

이제 ipython-sql 확장을 로드하고 데이터베이스와 연결을 설정하자

##### SQLite를 사용하여 magic sql에 연결하는 구문은 다음과 같다:

  **%sql sqlite://DatabaseName**   

여기서 DatabaseName은 **.db** 파일이 된다

In [8]:
import csv, sqlite3

con = sqlite3.connect("RealWorldData.db")
cur = con.cursor()

In [11]:

!pip install -q pandas==1.1.5

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 43.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  error: subprocess-exited-with-error
  
  × Building wheel for pandas (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for pandas
ERROR: ERROR: Failed to build installable wheels for some pyproject.toml based projects (pandas)


In [3]:
%load_ext sql

In [4]:
%sql sqlite:///RealWorldData.db

### 데이터 세트를 테이블에 저장하기

##### 분석할 데이터 세트는 대부분 .CSV(쉼표로 구분된 값) 파일 형식으로, 인터넷에서 제공되는 경우가 많다  
##### 데이터를 SQL로 분석하려면 먼저 데이터베이스에 저장해야 한다  

##### 먼저 주어진 URL에서 CSV 파일을 읽어 pandas 데이터프레임으로 변환할 것이다  

##### 이후 df.to_sql() 함수를 사용하여 각 CSV 파일을 SQLite의 테이블로 변환하고 해당 데이터를 로드할 것이다  


In [9]:
import pandas as pd
df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/FinalModule_Coursera_V5/data/ChicagoPublicSchools.csv")
df.to_sql("CHICAGO_PUBLIC_SCHOOLS_DATA", con, if_exists='replace', index=False, method="multi")


566

In [ ]:
df.head()

### 데이터베이스 시스템 카탈로그를 쿼리하여 테이블 메타데이터 가져오기

##### 스키마 내 모든 테이블 목록을 조회하고 SCHOOLS 테이블이 생성되었는지 확인함으로써 테이블 생성이 성공적으로 이루어졌는지 확인할 수 있다.

In [7]:
%config SqlMagic.style = '_DEPRECATED_DEFAULT'

In [ ]:
# type in your query to retrieve list of all tables in the database
%sql SELECT name FROM sqlite_master WHERE type='table'

 * sqlite:///RealWorldData.db
Done.


name
CHICAGO_PUBLIC_SCHOOLS_DATA


### 데이터베이스 시스템 카탈로그를 쿼리하여 열 메타데이터 가져오기

##### SCHOOLS 테이블에는 많은 열이 포함되어 있다. 이 테이블에는 몇 개의 열이 있을까?


In [8]:
# type in your query to retrieve the number of columns in the SCHOOLS table
%sql SELECT COUNT(name) FROM PRAGMA_TABLE_INFO('CHICAGO_PUBLIC_SCHOOLS_DATA');

 * sqlite:///RealWorldData.db
Done.


COUNT(name)
78


이제 SCHOOLS 테이블의 열 목록과 해당 열 유형(datatype) 및 길이를 조회하자.

In [9]:
# type in your query to retrieve all column names in the SCHOOLS table along with their datatypes and length
%sql SELECT name,type,length(type) FROM PRAGMA_TABLE_INFO('CHICAGO_PUBLIC_SCHOOLS_DATA');

 * sqlite:///RealWorldData.db
Done.


name,type,length(type)
School_ID,INTEGER,7
NAME_OF_SCHOOL,TEXT,4
"Elementary, Middle, or High School",TEXT,4
Street_Address,TEXT,4
City,TEXT,4
State,TEXT,4
ZIP_Code,INTEGER,7
Phone_Number,TEXT,4
Link,TEXT,4
Network_Manager,TEXT,4


### 질문

1. "SCHOOL ID" 속성의 열 이름은 대문자인가, 아니면 혼합 대소문자인가?  
2. 테이블에서 "Community Area Name" 열의 이름은 무엇인가? 공백이 포함되어 있는가?  
3. 열 이름에서 공백이나 괄호(둥근 괄호)가 밑줄 문자 "_"로 대체된 열이 있는가?  


## Problems

### Problem 1

##### How many Elementary Schools are in the dataset?
##### 데이터 세트에 포함된 초등학교의 수는 몇 개인가?

In [34]:
%sql select distinct count(*) from CHICAGO_PUBLIC_SCHOOLS_DATA where "Elementary, Middle, or High School"='ES'

 * sqlite:///RealWorldData.db
Done.


count(*)
462


### Problem 2

##### What is the highest Safety Score?
##### 가장 높은 Safety Score는?


In [32]:
%sql select max("SAFETY_SCORE") as'MAX_SAFETY_SCORE' from CHICAGO_PUBLIC_SCHOOLS_DATA

 * sqlite:///RealWorldData.db
Done.


MAX_SAFETY_SCORE
99.0


### Problem 3

##### Which schools have highest Safety Score?
##### 가장 높은 Safety Score를 가진 학교는 어떤 학교인가?

In [38]:
%sql select "NAME_OF_SCHOOL" ,"SAFETY_SCORE" from CHICAGO_PUBLIC_SCHOOLS_DATA where "SAFETY_SCORE"=(select max("SAFETY_SCORE")from CHICAGO_PUBLIC_SCHOOLS_DATA);

 * sqlite:///RealWorldData.db
Done.


NAME_OF_SCHOOL,SAFETY_SCORE
Abraham Lincoln Elementary School,99.0
Alexander Graham Bell Elementary School,99.0
Annie Keller Elementary Gifted Magnet School,99.0
Augustus H Burley Elementary School,99.0
Edgar Allan Poe Elementary Classical School,99.0
Edgebrook Elementary School,99.0
Ellen Mitchell Elementary School,99.0
James E McDade Elementary Classical School,99.0
James G Blaine Elementary School,99.0
LaSalle Elementary Language Academy,99.0


### Problem 4

##### What are the top 10 schools with the highest "Average Student Attendance"?
##### "Average Student Attendance"가 가장 높은 상위 10개 학교는 무엇인가?

In [44]:
%sql select "NAME_OF_SCHOOL","AVERAGE_STUDENT_ATTENDANCE" from CHICAGO_PUBLIC_SCHOOLS_DATA order by AVERAGE_STUDENT_ATTENDANCE desc limit 10;

 * sqlite:///RealWorldData.db
Done.


NAME_OF_SCHOOL,AVERAGE_STUDENT_ATTENDANCE
John Charles Haines Elementary School,98.40%
James Ward Elementary School,97.80%
Edgar Allan Poe Elementary Classical School,97.60%
Orozco Fine Arts & Sciences Elementary School,97.60%
Rachel Carson Elementary School,97.60%
Annie Keller Elementary Gifted Magnet School,97.50%
Andrew Jackson Elementary Language Academy,97.40%
Lenart Elementary Regional Gifted Center,97.40%
Disney II Magnet School,97.30%
John H Vanderpoel Elementary Magnet School,97.20%


### Problem 5

##### Retrieve the list of 5 Schools with the lowest Average Student Attendance sorted in ascending order based on attendance
##### Average Student Attendance가 가장 낮은 5개 학교를 조회하라. 출석률을 기준으로 오름차순 정렬하라.

In [46]:
%sql select "NAME_OF_SCHOOL", "AVERAGE_STUDENT_ATTENDANCE" from CHICAGO_PUBLIC_SCHOOLS_DATA order by AVERAGE_STUDENT_ATTENDANCE asc limit 5;

 * sqlite:///RealWorldData.db
Done.


NAME_OF_SCHOOL,AVERAGE_STUDENT_ATTENDANCE
Velma F Thomas Early Childhood Center,None
Richard T Crane Technical Preparatory High School,57.90%
Barbara Vick Early Childhood & Family Center,60.90%
Dyett High School,62.50%
Wendell Phillips Academy High School,63.00%


### Problem 6

##### Now remove the '%' sign from the above result set for Average Student Attendance column   
##### 이제 Average Student Attendance 열에서 '%' 기호를 제거하라.

In [49]:
%sql select "NAME_OF_SCHOOL", replace("AVERAGE_STUDENT_ATTENDANCE",'%','') from CHICAGO_PUBLIC_SCHOOLS_DATA order by AVERAGE_STUDENT_ATTENDANCE asc limit 5;

 * sqlite:///RealWorldData.db
Done.


NAME_OF_SCHOOL,"replace(""AVERAGE_STUDENT_ATTENDANCE"",'%','')"
Velma F Thomas Early Childhood Center,None
Richard T Crane Technical Preparatory High School,57.90
Barbara Vick Early Childhood & Family Center,60.90
Dyett High School,62.50
Wendell Phillips Academy High School,63.00


 * sqlite:///RealWorldData.db
Done.


NAME_OF_SCHOOL,"REPLACE(Average_Student_Attendance, '%', '')"
Velma F Thomas Early Childhood Center,None
Richard T Crane Technical Preparatory High School,57.90
Barbara Vick Early Childhood & Family Center,60.90
Dyett High School,62.50
Wendell Phillips Academy High School,63.00


Double-click **here** for a hint

<!--
Use the REPLACE() function to replace '%' with ''

-->


Double-click **here** for the solution.

<!-- Hint:

%sql SELECT Name_of_School, REPLACE(Average_Student_Attendance, '%', '') \
     from CHICAGO_PUBLIC_SCHOOLS_DATA \
     order by Average_Student_Attendance \
     LIMIT 5

-->


### Problem 7

##### Which Schools have Average Student Attendance lower than 70%?
##### Average Student Attendance가 70% 미만인 학교는 어떤 학교인가?

In [54]:
%sql select "NAME_OF_SCHOOL", "AVERAGE_STUDENT_ATTENDANCE" from CHICAGO_PUBLIC_SCHOOLS_DATA  where "AVERAGE_STUDENT_ATTENDANCE" <70 order by "AVERAGE_STUDENT_ATTENDANCE" asc ;

 * sqlite:///RealWorldData.db
Done.


NAME_OF_SCHOOL,AVERAGE_STUDENT_ATTENDANCE
Richard T Crane Technical Preparatory High School,57.90%
Barbara Vick Early Childhood & Family Center,60.90%
Dyett High School,62.50%
Wendell Phillips Academy High School,63.00%
Orr Academy High School,66.30%
Manley Career Academy High School,66.80%
Chicago Vocational Career Academy High School,68.80%
Roberto Clemente Community Academy High School,69.60%


Double-click **here** for a hint

<!--
The datatype of the "Average_Student_Attendance" column is varchar.
So you cannot use it as is in the where clause for a numeric comparison.
First use the CAST() function to cast it as a DECIMAL or DOUBLE
e.g. CAST("Column_Name" as DOUBLE)

-->


Double-click **here** for another hint

<!--
Don't forget the '%' age sign needs to be removed before casting
-->


Double-click **here** for the solution.

<!-- Solution:

%sql SELECT Name_of_School, Average_Student_Attendance  \
     from CHICAGO_PUBLIC_SCHOOLS_DATA \
     where CAST ( REPLACE(Average_Student_Attendance, '%', '') AS DOUBLE ) < 70 \
     order by Average_Student_Attendance
     

-->


### Problem 8

##### Get the total College Enrollment for each Community Area
##### 각 Community Area별 총 College Enrollment를 구하라.

In [68]:
%sql select "COMMUNITY_AREA_NAME", sum("COLLEGE_ENROLLMENT")as"TOTAL_ENROLLMENT"  from CHICAGO_PUBLIC_SCHOOLS_DATA group by "COMMUNITY_AREA_NAME" ;

 * sqlite:///RealWorldData.db
Done.


COMMUNITY_AREA_NAME,TOTAL_ENROLLMENT
ALBANY PARK,6864
ARCHER HEIGHTS,4823
ARMOUR SQUARE,1458
ASHBURN,6483
AUBURN GRESHAM,4175
AUSTIN,10933
AVALON PARK,1522
AVONDALE,3640
BELMONT CRAGIN,14386
BEVERLY,1636


### Problem 9

##### Get the 5 Community Areas with the least total College Enrollment  sorted in ascending order
##### 총 College Enrollment가 가장 적은 상위 5개 Community Area를 구하라. 오름차순으로 정렬하라.

In [10]:
%sql select "COMMUNITY_AREA_NAME","COLLEGE_ENROLLMENT" from CHICAGO_PUBLIC_SCHOOLS_DATA order by "COLLEGE_ENROLLMENT" asc ;

 * sqlite:///RealWorldData.db
Done.


KeyError: 'DEFAULT'

### Problem 10

##### List 5 schools with lowest safety score.
##### Safety Score가 가장 낮은 상위 5개 학교를 나열하라.

 * sqlite:///RealWorldData.db
Done.


NAME_OF_SCHOOL,SAFETY_SCORE
Edmond Burke Elementary School,1.0
Luke O'Toole Elementary School,5.0
George W Tilton Elementary School,6.0
Foster Park Elementary School,11.0
Emil G Hirsch Metropolitan High School,13.0


Double-click **here** for hint.

<!-- Solution SQLite:
safety_score 가 'None'이 아닐 조건을 추가해야 해요.
where safety_score !='None'

-->




## 요약

##### 이 실습에서 SQL과 Python을 사용하여 실제 데이터 세트를 다루는 방법을 배웠다. 공백이나 특수 문자가 포함된 열 이름 및 혼합 대소문자로 구성된 열 이름을 쿼리하는 방법을 배웠다. 또한, 데이터베이스 내장 함수를 사용하고, 결과 집합을 정렬, 제한, 순서 지정하는 방법을 연습했으며, 서브쿼리를 사용하고 여러 테이블을 다루는 방법도 배웠다.
